In [65]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase
import os
import langchain_core

In [66]:
from Flemmings_Library.variables import db_path, db
from Flemmings_Library.ollama import get_ollama_model_names

!ollama list > nul 2>&1 # Für Windows: Ollama starten, Ausgabe der Modelle unterdrücken

In [67]:
model_names = get_ollama_model_names(debug=False)
db_dialect = db.dialect
model_name = model_names[0] # Modell festlegen

In [68]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
)
print(f'{model_name} ausgewählt und geladen.')

llama3.2-vision:11b ausgewählt und geladen.


Schema der Datenbank extrahieren

In [69]:
from Flemmings_Library.extract import extract_schema

schema = extract_schema(db_path, sample_rows=1, verbose=False)
schema_info_str = str(schema)

State-Klasse definieren

In [70]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict

class State(TypedDict):
    schema_info: str
    prompt: str
    question: str      
    db_dialect: str
    query: str           
    execution_result: str
    error: str

Prompt-Template erstellen und State initiieren

In [71]:
question = "In which city is the customer with the highest customer ID located?"

In [72]:
from Flemmings_Library.variables import chat_prompt_template_sql

In [73]:
formatted_prompt = chat_prompt_template_sql.format(
    schema_info=schema_info_str,
    question=question,
    db_dialect=db_dialect
)
print(formatted_prompt)


System: You are a helpful SQL assistant that provides only SELECT-Statements. Based on the following database schema, translate a natural language query into an SQL query that is executable in my sqlite-Database.

Database Schema:
{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embrae

In [74]:
# initialen State festlegen
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db.dialect,
    "prompt": formatted_prompt,
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

Query durch LLM ausgeben lassen und ausführen

In [75]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt_template_sql | llm | StrOutputParser()

In [76]:
llm_output = chain.invoke({
            "schema_info": schema_info_str,
            "question": question,
            "db_dialect": db.dialect
        })

state["query"] = llm_output
state['execution_result'] = db.run(state['query'])

In [77]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe

Antwort des Modells in Textform

In [79]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

In [81]:
answer_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful SQL assistant. Based on the SQL query and its execution result, provide a concise final answer to the user's question."
    ),
    HumanMessagePromptTemplate.from_template(
        "User Question: {question}\n"
        "SQL Query: {query}\n"
        "Execution Result: {execution_result}\n\nPlease provide your final answer:"
    )
])

In [83]:
formatted_answer_prompt = answer_prompt_template.format(
    question=state["question"],
    query=state["query"],
    execution_result=state["execution_result"]
)

print(formatted_answer_prompt)

System: You are a helpful SQL assistant. Based on the SQL query and its execution result, provide a concise final answer to the user's question.
Human: User Question: In which city is the customer with the highest customer ID located?
SQL Query: SELECT City FROM Customer WHERE CustomerId = ( SELECT MAX(CustomerId) FROM Customer )
Execution Result: [('Bangalore',)]

Please provide your final answer:


In [86]:
chain_answer = answer_prompt_template | llm | StrOutputParser()

In [88]:
llm_answer = chain_answer.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "query": state["query"],
            "execution_result": state["execution_result"]
        })

state["answer"] = llm_answer

In [89]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe